### SNIPS NLU

#### credit -> https://colab.research.google.com/drive/1wgWdxUpKf3FWJgqA6ogBGDEzxAosjJMI

Snips NLU consists of 2 tasks (Slot Filling and Classification)

Slot filling can be formulated as NER

### Download Data

In [ ]:
from urllib.request import urlretrieve
from pathlib import Path


SNIPS_DATA_BASE_URL = (
    "https://github.com/ogrisel/slot_filling_and_intent_detection_of_SLU/blob/"
    "master/data/snips/"
)
for filename in ["train", "valid", "test", "vocab.intent", "vocab.slot"]:
    path = Path(filename)
    if not path.exists():
        print(f"Downloading {filename}...")
        urlretrieve(SNIPS_DATA_BASE_URL + filename + "?raw=true", path)

In [1]:
# Read SNIPS data

import pandas as pd
import numpy as np
from pathlib import Path


def parse_line(line):
    utterance_data, intent_label = line.split(" <=> ")
    items = utterance_data.split()
    words = [item.rsplit(":", 1)[0] for item in items]
    word_labels = [item.rsplit(":", 1)[1] for item in items]
    return {
        "intent_label": intent_label,
        "words": " ".join(words),
        "word_labels": " ".join(word_labels),
        "length": len(words),
    }


lines_train = Path("train").read_text().strip().splitlines()
lines_valid = Path("valid").read_text().strip().splitlines()
lines_test = Path("test").read_text().strip().splitlines()

df_train = pd.DataFrame([parse_line(line) for line in lines_train])
df_valid = pd.DataFrame([parse_line(line) for line in lines_valid])
df_test = pd.DataFrame([parse_line(line) for line in lines_test])

# Slot labels
slot_names = ["[PAD]"]
slot_names += Path("vocab.slot").read_text().strip().splitlines()
slot_map = {}
for label in slot_names:
    slot_map[label] = len(slot_map)

In [2]:
df_train

,intent_label,words,word_labels,length
0,AddToPlaylist,Add Don and Sherri to my Meditate to Sounds of...,O B-entity_name I-entity_name I-entity_name O ...,12
1,AddToPlaylist,put United Abominations onto my rare groove pl...,O B-entity_name I-entity_name O B-playlist_own...,8
2,AddToPlaylist,add the tune by misato watanabe to the Trapeo ...,O O B-music_item O B-artist I-artist O O B-pla...,10
3,AddToPlaylist,add this artist to my this is miguel bosé play...,O O B-music_item O B-playlist_owner B-playlist...,10
4,AddToPlaylist,add heresy and the hotel choir to the evening ...,O B-entity_name I-entity_name I-entity_name I-...,11
...,...,...,...,...
13079,SearchScreeningEvent,find a Consolidated Theatres showing The Good ...,O O B-location_name I-location_name O B-movie_...,10
13080,SearchScreeningEvent,where can i see animated movies in the neighbo...,O O O O B-movie_type I-movie_type B-spatial_re...,9
13081,SearchScreeningEvent,Showtimes for animated movies in the area .,O O B-movie_type I-movie_type B-spatial_relati...,8
13082,SearchScreeningEvent,Which animated movies are playing at Megaplex ...,O B-movie_type I-movie_type O O O B-location_n...,11


In [ ]:
from transformers import AlbertTokenizer
from tf_transformers.losses import cross_entropy_loss_fast
from tf_transformers.core import optimization
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
SPECIAL_PIECE = "▁"

In [4]:


def get_tokens_labels(
    aligned_words, orig_to_new_index, label_tokens, sub_words_mapped, label_pad_token="[PAD]"
):
    """
    convert each sub word into labels
    If a word is split into multiple sub words, 
    then first sub word is assigned with label and other sub words will be padded 
    """
    aligned_labels = [label_pad_token] * len(aligned_words)
    for original_pos, new_pos in enumerate(orig_to_new_index):
        aligned_labels[new_pos] = label_tokens[original_pos]
    flat_tokens = []
    flat_labels = []

    # The first word of the subword token is assigned entity
    # other tokens will be add PAD labels (we will mask it while training)
    assert (len(aligned_words) == len(sub_words_mapped) == len(aligned_labels))
    for (_align_word, _align_word, _align_label) in zip(
        aligned_words, sub_words_mapped, aligned_labels
    ):
        temp_w = []
        for _align_word in _align_word:
            temp_w.append(_align_word)
        temp_l = [label_pad_token] * len(temp_w)
        temp_l[0] = _align_label
        flat_tokens.extend(temp_w)
        flat_labels.extend(temp_l)
        
    return flat_tokens, flat_labels


# from tf_transformers.utils import fast_sp_alignment

def fast_sp_split(sentence, tokenizer, SPECIAL_PIECE):
    original_words = sentence.split()
    subwords = tokenizer.tokenize(sentence)

    # Convert text into main_words (list of list of subwords per word)
    sub_words_mapped = []
    temp_tokens = []
    for tok in subwords:
        if tok == SPECIAL_PIECE:
            if temp_tokens:
                sub_words_mapped.append(temp_tokens)
                temp_tokens = []
            sub_words_mapped.append([tok])

        else:
            if tok.startswith(SPECIAL_PIECE):
                if temp_tokens:
                    sub_words_mapped.append(temp_tokens)
                    temp_tokens = []
                temp_tokens.append(tok)
            else:
                temp_tokens.append(tok)

    if temp_tokens:
        sub_words_mapped.append(temp_tokens)
    return original_words, sub_words_mapped

def account_for_special_piece_local(
    word_tokens, sub_words_mapped
):
    # this loop is used to accout for extra SPECIAL_PIECE
    # if any
    special_counter = 0
    aligned_words = []
    orig_to_new_index = []
    for index, _sub_word in enumerate(sub_words_mapped):
        # this is some extra SPECIAL PIECE character
        # add it to original word
        if len(_sub_word) == 1 and _sub_word[0] == SPECIAL_PIECE:
            special_counter += 1
            aligned_words.append(_sub_word[0])
        else:
            pos = index - special_counter
            aligned_words.append(word_tokens[pos])
            orig_to_new_index.append(index) # whenever original words comes, we need old-new mapping
    return orig_to_new_index, aligned_words

def fast_sp_alignment(sentence, tokenizer, SPECIAL_PIECE):
    """Fast Sentence Piece Alignment

    A sentence will be split into tokens based on whitespace, then tokenize using
    sentence piece tokenizer (GPT2, Albert, etc).

    Args:
        sentence ([type]): [description]
        tokenizer ([type]): [description]
        SPECIAL_PIECE ([type]): [description]

    Returns:
        [orig_to_new_index]: [list: Old to new index mapping alignment]
        [aligned_words]: [list of string: Aligns words (by adding SPECIAL_PIECE) if required]
        [sub_words_mapped]: [list of list of subwords]
    """

    original_words, sub_words_mapped = fast_sp_split(sentence, tokenizer, SPECIAL_PIECE)
    # If they are of different length mostly due to
    # extra SPECIAL_PIECE or unicode characters
    if len(original_words) != len(sub_words_mapped):
        # Try to re-align if possible
        try:
            orig_to_new_index, aligned_words = account_for_special_piece_local(original_words, sub_words_mapped)
        except:
            # if re-align fails, then tokenize like word-piece tokenizer
            # but, using sentence piece
            aligned_words = original_words
            sub_words_mapped = [tokenizer.tokenize(word) for word in original_words]
            orig_to_new_index = range(len(original_words))

        assert(len(aligned_words) == len(sub_words_mapped))
        return orig_to_new_index,  aligned_words, sub_words_mapped
    else:
        # If this mapping fails, logic fails
        orig_to_new_index = range(len(original_words))
        return orig_to_new_index, original_words, sub_words_mapped


# new_to_orig_dict = dict(zip(orig_to_new_index, range(len(orig_to_new_index))))
# # reverse mapping

# reverse_start_index_align = tok_to_orig_index[tok_start_position] # aligned index
# reverse_end_index_align   = tok_to_orig_index[tok_end_position]


# reverse_start_index_original = new_to_orig_dict[reverse_start_index_align] # original (doc_tokens) index
# reverse_start_index_stop = new_to_orig_dict[reverse_end_index_align]

def tokenize_and_align_sentence_label(
    sentence, word_tokens, label_tokens, label_pad_token
):
        
    """
    align sentence sub words and labels using fast_sp
    """
    subwords = tokenizer.tokenize(sentence)
    orig_to_new_index, aligned_words, sub_words_mapped = fast_sp_alignment(
            sentence, tokenizer, SPECIAL_PIECE
        )
    
    flat_tokens, flat_labels = get_tokens_labels(aligned_words, orig_to_new_index, label_tokens, sub_words_mapped, label_pad_token
    )
    return flat_tokens, flat_labels

def tokenize_and_align_sentence_label_valid(
    sentence, word_tokens, label_tokens, label_pad_token
):
        
    """
    align sentence sub words and labels using fast_sp
    """
    subwords = tokenizer.tokenize(sentence)
    orig_to_new_index, aligned_words, sub_words_mapped = fast_sp_alignment(
            sentence, tokenizer, SPECIAL_PIECE
        )
    
    flat_tokens, flat_labels = get_tokens_labels(aligned_words, orig_to_new_index, label_tokens, sub_words_mapped, label_pad_token
    )
    return aligned_words, sub_words_mapped, flat_tokens, flat_labels


encoder_max_length = 50  # 50 is enough for SNIPS
def process_data_to_model_inputs(sentence, flat_labels, label_pad_token):
    """
    convert text to inputs (ids)
    """
    # Tokenizer will automatically set [BOS] <text> [EOS]
    result = {}
    result["input_ids"] = tokenizer.encode(
        sentence, truncation=True, max_length=encoder_max_length
    )
    result["input_mask"] = [1] * len(result["input_ids"])
    result["input_type_ids"] = [0] * len(result["input_ids"])
    labels = [slot_map[token] for token in flat_labels]
    labels = [slot_map[label_pad_token]] + labels + [slot_map[label_pad_token]]  # for [CLS] and [SEP]
    label_mask = []
    for token in flat_labels:
        if token == [label_pad_token]:
            label_mask.append(0)
            continue
        label_mask.append(1)
    label_mask = [0] + label_mask + [0]  # for [CLS] and [SEP]
    result["labels"] = labels
    result["label_mask"] = label_mask
    return result


ignored_index = []
mysample = []
def make_parse_fn(df):
    """
    Iterte over data frame
    
    convert (text, labels) -> sub words and labels for each sub_words
    
    """
    for index, row in df.iterrows():
        sentence = row["words"]
        labels = row["word_labels"]
        word_tokens = sentence.split()
        label_tokens = labels.split()
        if len(word_tokens) != len(label_tokens):
            ignored_index.append(index)
            continue
        aligned_words_new, main_words_new, flat_tokens, flat_labels = tokenize_and_align_sentence_label_valid(
            sentence, word_tokens, label_tokens, label_pad_token="[PAD]")
        mysample.append((flat_tokens, flat_labels))
        yield process_data_to_model_inputs(sentence, flat_labels, label_pad_token="[PAD]")
    print("Ignored {} indexes".format(len(ignored_index)))




In [5]:
# Sample sentence
sentence = 'I love to listen to Carnatic songs by K.J.Yesudas'
labels   = 'O O O O O I-genre O O I-artist'
word_tokens = sentence.split()
label_tokens = labels.split()
assert(len(word_tokens) == len(label_tokens))
flat_tokens, flat_labels = tokenize_and_align_sentence_label(
    sentence, word_tokens, label_tokens, label_pad_token="[PAD]"
)
sample_inputs = process_data_to_model_inputs(sentence, flat_labels, label_pad_token="[PAD]")

NameError: name 'tokenizer' is not defined

In [ ]:
# use TFProcessor only if your data is in range of 10k - 20k maximum
# otherwise use TFWriter
from tf_transformers.data import TFProcessor

tf_processor = TFProcessor()
parse_fn = make_parse_fn(df_train)
train_dataset = tf_processor.process(parse_fn)

parse_fn_valid = make_parse_fn(df_valid)
valid_dataset = tf_processor.process(parse_fn_valid)

In [ ]:
import tensorflow as tf
def ragged_dict_to_tensor(x):
    x_temp = {}
    for k,v in x.items():
        x_temp[k] = v.to_tensor()
    return x_temp

def separate_x_y(input_dict):
    x = {}
    y = {}
    for k,v in input_dict.items():
        if k in x_keys:
            x[k] = v
        else:
            y[k] = v
    return (x, y)

x_keys = ['input_ids', 'input_type_ids', 'input_mask']
y_keys = ['labels', 'label_mask']
batch_size = 32
dataset = train_dataset.batch(batch_size)
dataset = dataset.map(ragged_dict_to_tensor)
dataset = dataset.map(separate_x_y)

dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [47]:
# Lets load Albert Model
from tf_transformers.models import AlbertModel
from tf_transformers.core import optimization

from absl import logging
logging.set_verbosity("INFO")

model_layer, model, config = AlbertModel(model_name='albert_base_v2', 
                   is_training=True, 
                   use_dropout=False
                   )
model.load_checkpoint("/mnt/home/PRE_MODELS/LegacyAI_models/checkpoints/albert-base-v2/")
del model_layer

INFO:absl:Initialized Variables


In [ ]:
# model.load_checkpoint("your checkpoint dir")

# Please have a look at tf_transformers/extra/*.py for reference values

input_ids  = tf.constant([[1, 9, 10, 11, 23], 
                         [1, 22, 234, 432, 2349]])
input_mask = tf.ones_like(input_ids)
input_type_ids = tf.ones_like(input_ids)

inputs = {'input_ids': input_ids, 
          'input_mask': input_mask, 
          'input_type_ids': input_type_ids}

results_tf_transformers   = model(inputs)
for k, r in results_tf_transformers.items():
    if isinstance(r, list):
        continue
    print(k, '-->', tf.reduce_sum(r), '-->', r.shape)

In [73]:
from tf_transformers.core import LegacyModel, LegacyLayer


class Token_Classification(LegacyLayer):
    def __init__(self, model, token_vocab_size, use_all_layers=False, activation=None, **kwargs):
        super(Token_Classification, self).__init__(**kwargs)
        self.model = model
        if isinstance(model, LegacyModel):
            self.model_config = model.model_config
        elif isinstance(model, tf.keras.layers.Layer):
            self.model_config = model._config_dict
        self.use_all_layers = use_all_layers
        self.logits_layer = tf.keras.layers.Dense(
            token_vocab_size,
            activation=activation
        )

    def call(self, inputs):
        result = self.model(inputs)
        token_logits = []
        if self.use_all_layers:
            # each layer token embeddings
            for token_embeddings in result["all_layer_token_embeddings"]:
                outputs = self.logits_layer(token_embeddings)
                token_logits.append(outputs)
            return {'token_logits': token_logits}            

        else:
            # last layer token embeddings
            token_embeddings = result["token_embeddings"]
            outputs = self.logits_layer(token_embeddings)
            return {
                    "token_logits": outputs
            }
        
    def get_model(self):
        layer_output = self(self.model.input)
        model = LegacyModel(inputs=self.model.input, outputs=layer_output, name='token_classification')
        model.model_config = self.model_config
        return model
        

In [74]:
tf.keras.backend.clear_session()
model_ner = Token_Classification(model=model,
                                      token_vocab_size=len(slot_map),
                                      use_all_layers=True, 
                                      is_training=True)
model_ner = model_ner.get_model()

In [84]:
def token_loss(y_true_dict, token_logits):
    loss = cross_entropy_loss_fast(
        labels=y_true_dict["labels"],
        logits=token_logits,
        label_weights=y_true_dict["label_mask"],
    )
    return loss

def token_loss_all_layers(y_true_dict, y_pred_dict):
    layer_loss = []
    for token_logits in y_pred_dict['token_logits']:
        loss = token_loss(y_true_dict, token_logits)
        layer_loss.append(loss)
    return tf.reduce_mean(layer_loss)

In [ ]:
train_data_size = 13000
learning_rate   = 2e-5
steps_per_epoch = int(train_data_size / batch_size)
EPOCHS = 3
num_train_steps = steps_per_epoch * EPOCHS
warmup_steps = int(EPOCHS * train_data_size * 0.1 / batch_size)
# creates an optimizer with learning rate schedule
optimizer_type = 'adamw'
optimizer, learning_rate_fn = optimization.create_optimizer(learning_rate,
                                                steps_per_epoch * EPOCHS,
                                                warmup_steps,
                                                optimizer_type)

# # Compile
# optimizer = tf.keras.optimizers.Adam()
# loss_fn = {'token_logits': token_loss_all_layers}
# model_ner.compile2(optimizer=optimizer, 
#                             loss=None, 
#                             custom_loss=loss_fn)



# class LossHistory(tf.keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []
#         self.val_losses = []

#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))
#         self.val_losses.append(logs.get('val_loss'))
# # Fit
# history_callbacks = LossHistory()
# history = model_ner.fit(dataset, epochs=3, callbacks=[history_callbacks])

In [ ]:
slot_map_reverse = {v:k for k,v in slot_map.items()}

In [ ]:
model_ner.save_checkpoint("temp_model", overwrite=True)
model_ner = Token_Classification(model=model,
                                      token_vocab_size=len(slot_map),
                                      use_all_layers=True, 
                                      is_training=False)
model_ner = model_ner.get_model()
model_ner.load_checkpoint("temp_model")


batch_size = 5
dataset_valid = valid_dataset.batch(batch_size)
dataset_valid = dataset_valid.map(ragged_dict_to_tensor)
dataset_valid = dataset_valid.map(separate_x_y)

num_layers = 12
prediction_per_layer = {i:[] for i in range(num_layers)}
original_labels = []
for (batch_inputs, batch_labels) in dataset_valid:
    results = model_ner(batch_inputs)
    model_logits = results['token_logits'][-1]
    
    for i, model_logits in enumerate(results['token_logits']):
    
        # Iterate over each example
        for index, per_example_logits in enumerate(model_logits):
            per_example_length = tf.reduce_sum(batch_inputs['input_mask'][index])
            per_example_label  = batch_labels['labels'][index][:per_example_length][1:-1] # we dont want pad positions and 1:-1 is to remove CLS and SEP
            per_example_logits = per_example_logits[:per_example_length][1:-1] # 1:-1 CLS and SEP
            per_example_preds  = tf.argmax(per_example_logits, axis=-1)
            prediction_per_layer[i].append(per_example_preds)
            
            # We want the original label only once
            if i == 0:
                original_labels.append(per_example_label)
    
    
# We have 700 examples
for layer_iter in range(num_layers):
    result = prediction_per_layer[layer_iter]
    
    pred_list = []
    for i in range(700):
        pred = list(result[i].numpy())
        orig = list(original_labels[i].numpy())
        if pred == orig:
            pred_list.append(1)
        else:
            pred_list.append(0)
    print("Layer {} exact match {} / 700".format(layer_iter, sum(pred_list)))
    
# Layer 0 exact match 110 / 700
# Layer 1 exact match 321 / 700
# Layer 2 exact match 436 / 700
# Layer 3 exact match 500 / 700
# Layer 4 exact match 531 / 700
# Layer 5 exact match 553 / 700
# Layer 6 exact match 567 / 700
# Layer 7 exact match 575 / 700
# Layer 8 exact match 570 / 700
# Layer 9 exact match 573 / 700
# Layer 10 exact match 572 / 700
# Layer 11 exact match 566 / 700

In [ ]:
# So, 5 the layer itself is giving 531/700 
# Thats great. Lets finalize the model with 5 hidden layers (instead of 12)

from tf_transformers.models import AlbertEncoder

config_copy = config.copy()
config_copy['num_hidden_layers'] = 5
model_final = AlbertEncoder(config, 
                            mask_mode=config['mask_mode'], 
                            is_training=False, 
                            name='albert')
model_final  = model_final.get_model()
model_final.load_checkpoint("/mnt/home/PRE_MODELS/LegacyAI_models/checkpoints/albert-base-v2/")

# As we finalized the best layer
# we use_all_layers=False
tf.keras.backend.clear_session()
model_ner_final = Token_Classification(model=model_final,
                                      token_vocab_size=len(slot_map),
                                      use_all_layers=False, 
                                      is_training=False)
model_ner_final = model_ner_final.get_model()
model_ner_final.load_checkpoint("temp_model")


predictions_final = []
original_labels = []
for (batch_inputs, batch_labels) in dataset_valid:
    results = model_ner_final(batch_inputs)
    model_logits = results['token_logits']
    
    
    # Iterate over each example
    for index, per_example_logits in enumerate(model_logits):
        per_example_length = tf.reduce_sum(batch_inputs['input_mask'][index])
        per_example_label  = batch_labels['labels'][index][:per_example_length][1:-1] # we dont want pad positions and 1:-1 is to remove CLS and SEP
        per_example_logits = per_example_logits[:per_example_length][1:-1] # 1:-1 CLS and SEP
        per_example_preds  = tf.argmax(per_example_logits, axis=-1)
        
        predictions_final.append(per_example_preds)
        original_labels.append(per_example_label)
        
# Evaluate  
pred_list = []
for i in range(700):
    pred = list(predictions_final[i].numpy())
    orig = list(original_labels[i].numpy())
    if pred == orig:
        pred_list.append(1)
    else:
        pred_list.append(0)
print("exact match {} / 700".format(sum(pred_list)))

# It matches with the result of layer 5 (actualy layer 4 if we start from index 0)
# exact match 566 / 700

In [ ]:
# Serialize the model
model_ner_final.save_as_serialize_module("temp_model_pb")

In [ ]:
# Convert to tflite

# Make sure none of the input dimension is None
model_final = AlbertEncoder(config, 
                            mask_mode=config['mask_mode'], 
                            is_training=False,
                            batch_size=1,
                            sequence_length=50, # 50 is enough for SNIPS
                            name='albert')
model_final  = model_final.get_model()
model_final.load_checkpoint("/mnt/home/PRE_MODELS/LegacyAI_models/checkpoints/albert-base-v2/")

# As we finalized the best layer
# we use_all_layers=False
tf.keras.backend.clear_session()
model_ner_final = Token_Classification(model=model_final,
                                      token_vocab_size=len(slot_map),
                                      use_all_layers=False, 
                                      is_training=False)
model_ner_final = model_ner_final.get_model()
model_ner_final.load_checkpoint("temp_model")
model_ner_final.save_as_serialize_module("temp_model_pb_tflite")

converter = tf.lite.TFLiteConverter.from_saved_model("temp_model_pb_tflite") # path to the SavedModel directory
converter.experimental_new_converter = True

tflite_model = converter.convert()

open("converted_model.tflite", "wb").write(tflite_model)


In [ ]:
#   Production

# Load saved model

loaded = tf.saved_model.load("temp_model_pb")
model = loaded.signatures['serving_default']


sentence = 'I love to listen to Carnatic songs by K.J.Yesudas'
labels   = 'O O O O O I-genre O O I-artist'
word_tokens = sentence.split()
label_tokens = labels.split()
assert(len(word_tokens) == len(label_tokens))
aligned_words, main_words, flat_tokens, flat_labels = tokenize_and_align_sentence_label_valid(
        sentence, word_tokens, label_tokens, label_pad_token="[PAD]"
    )
sample_inputs = process_data_to_model_inputs(sentence, flat_labels, label_pad_token="[PAD]")
sample_inputs = {k: tf.constant([v]) for  k, v in sample_inputs.items() if k in ['input_ids', 'input_mask', 'input_type_ids']}

results = model(**sample_inputs)

slot_logits = results['token_logits']
slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1] # to avoid CLS and SEP
slot_probs = slot_logits.numpy().max(axis=-1)[0, 1:-1]

# Extract only words that we want
subword_counter = -1
predicted_ids = []
for sub_word_list in main_words:
    if len(sub_word_list) == 1 and sub_word_list[0] == SPECIAL_PIECE:
        subword_counter += 1
        continue
    else:
        predicted_ids.append(slot_ids[subword_counter+1])
        subword_counter += len(sub_word_list)

predicted_labels = [slot_map_reverse[idx] for idx in predicted_ids]



In [ ]:
# Check same model with tflite

import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

sample_inputs = process_data_to_model_inputs(sentence, flat_labels, label_pad_token="[PAD]")

interpreter.set_tensor(input_details[0]['index'], tf.concat([sample_inputs['input_ids'],
                                                             tf.zeros((1, 31), dtype=tf.int32)], axis=1))

interpreter.set_tensor(input_details[1]['index'], tf.concat([sample_inputs['input_mask'],
                                                             tf.zeros((1, 31), dtype=tf.int32)], axis=1))

interpreter.set_tensor(input_details[2]['index'], tf.concat([sample_inputs['input_type_ids'],
                                                             tf.zeros((1, 31), dtype=tf.int32)], axis=1))

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
output_data = np.squeeze(output_data, 0)[:18, :]

slot_ids = output_data.argmax(axis=-1)
slot_probs = output_data.max(axis=-1)

# Extract only words that we want
subword_counter = -1
predicted_ids = []
for sub_word_list in main_words:
    if len(sub_word_list) == 1 and sub_word_list[0] == SPECIAL_PIECE:
        subword_counter += 1
        continue
    else:
        predicted_ids.append(slot_ids[subword_counter+1])
        subword_counter += len(sub_word_list)

predicted_labels = [slot_map_reverse[idx] for idx in predicted_ids]

